In [1]:
from othello import Othello
import numpy as np
from tqdm import tqdm

In [2]:
#seed 9 for train, 10 for test

def generate_data(data_size, seed):
    rng = np.random.default_rng(seed)
    moves = np.zeros((data_size,60), dtype=int)
    for run in tqdm(range(data_size)):
        game = Othello()
        for i in range(60):
            valid_moves = game.valid_moves()
            if not valid_moves:
                game.move_pass()
                moves[run,i]=60
            else:
                move = rng.choice(valid_moves)
                game.move(*move)
                moves[run,i]=Othello.pos_to_token(*move)

    return moves

In [ ]:
np.save('./data/moves.npy', generate_data(100000,9))

In [4]:
np.save('./data/test_moves.npy', generate_data(100,10))
#np.save('large_val.npy', generate_data(4096, 1000))

100%|██████████| 100/100 [00:01<00:00, 67.32it/s]


In [4]:
# rearranged to allow multiprocessing to speed up generation; runs on linux but hangs in windows - think run_game would have to be in another file for windows

def run_game(run):
    game = Othello()
    moves = np.zeros((60,), dtype=int)
    rng = np.random.default_rng(9 + 25*run)
    for i in range(60):
        
        valid_moves = game.valid_moves()
        if not valid_moves:
            game.move_pass()
            moves[i]=60
        else:
            move = rng.choice(valid_moves)
            game.move(*move)
            moves[i]=Othello.pos_to_token(*move)
    return moves

In [5]:
from multiprocessing import Pool

pool = Pool()

results = list(tqdm(pool.imap_unordered(run_game, range(1048576)), total = 1048576))
pool.close()
pool.join()

#np.save('large_training.npy', np.array(results))

  0%|          | 0/100 [00:00<?, ?it/s]